In [2]:
import sys
print(sys.version)

3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 08:20:52) 
[GCC 7.3.0]


In [3]:
import pytorch_lightning
pytorch_lightning.__version__

'0.7.6'

In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
sys.path.append("/nfs/homedirs/herrmanp/clevertest/adversarial-robustness-toolbox")
import art

In [6]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [7]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Train Model

In [8]:
from pytorch_lightning.callbacks import Callback

class SaveCallback(Callback): 
    def __init__(self, model_name = "newest_model"):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            save_path = self.model_name + "_v{}".format(trainer.logger.version) +  "_best.p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveCallback("iiii")

In [26]:
# best M5: batchsize 32, lr 9e-4, weithgdecay 0.001, lr decay 0.85
import config
import pytorch_lightning as pl
from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.CRNN import AudioCRNNPLModule

from attacks.pgd import ProjectedGradientDescent

print("Currently Best:", cb.best_val_acc)

hparams = {
    "batch_size": 64,
    "learning_rate": 9e-4,
    "weight_decay": 1e-3,
    "lr_decay": 0.85,
    "epochs": 2
}

model = CRNNPLModule(hparams)
model.prepare_data()
model = AudioCRNNPLModule(hparams, config=cfg).to(device)
model.float()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

# to train with gaussian noise:
#model.set_smooth(50)

trainer = pl.Trainer(
    max_epochs=hparams["epochs"],
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model).__name__),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

ImportError: cannot import name 'AudioCRNNPLModule' from 'classification.models.CRNN' (/nfs/homedirs/herrmanp/CURRENT/project-4/src/classification/models/CRNN.py)

In [18]:
!pip install git+https://github.com/ksanjeevan/torchparse.git

  Cloning https://github.com/ksanjeevan/torchparse.git to /tmp/pip-req-build-oj258i3k
  Running command git clone -q https://github.com/ksanjeevan/torchparse.git /tmp/pip-req-build-oj258i3k
  Created wheel for torchparse: filename=torchparse-0.1-py3-none-any.whl size=7977 sha256=4adbea6f0f06d2a719840b8b8b69f559db9786bd83f3ec33d6a4b0c4e4360317
  Stored in directory: /tmp/pip-ephem-wheel-cache-vilyy0qq/wheels/14/99/39/34754e0ce89e0a21e18986098cbc8346e7346b3cb7ce839129
Successfully built torchparse


In [10]:
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/optuna_90.5.p", SpectrogramCNNPLModule)
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p", M5PLModule)
model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/M5/M5_attack_num_iters_v99.p", M5PLModule)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.cuda()
model.report()

Accuracy: 	0.70
Precision: 	0.73
Recall: 	0.62
F1-Score: 	0.67

VAL-ACC: 1187/1687 (70.36158861885004%)

P-Rate: 	0.42


{'tp': array(514),
 'fp': array(188),
 'tn': array(673),
 'fn': array(312),
 'correct': array(1187),
 'n': 1687,
 'acc': 0.7036158861885004,
 'prec': 0.7321937321937322,
 'rec': 0.6222760290556901,
 'f1': 0.6727748691099477,
 'attack_args': None,
 'p_rate': 0.4161232957913456}

### Run a single attack

In [12]:
from attacks.volume import VolumeAttack
from attacks.pitch import PitchAttack
from attacks.speed import TimeStretchAttack
from attacks.pgd import ProjectedGradientDescent

attack = VolumeAttack(model.model, model.val_dataloader(), {"epsilon":0.9}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

100%|██████████| 27/27 [00:03<00:00,  6.92it/s]

Attack-Parameters:	{'epsilon': 0.9}
Early stopping: 	False (-1)

Successfully attacked:	686
Total attacked: 	1184
Total processed:	1687

Success-Rate: 		0.58
Perturbed Accurracy: 	0.3


{'success_rate': 0.5793918918918919, 'acc': 0.2951985773562537}

In [15]:
attack.showAdversarialExample(target_class=1)

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform


In [29]:
hparams = {
    "batch_size": 24,
    "learning_rate": 0.002,
    "weight_decay": 0.01,
    "lr_decay": 1
}

model = CRNNPLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

trainer = pl.Trainer(
    max_epochs=20,
    #logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model)._name_),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

RuntimeError: Expected object of scalar type Long but got scalar type Float for sequence element 1 in sequence argument at position #1 'tensors'